# Crear la clase  Target



In [1]:
import duckdb
import pandas as pd
import os
import time

In [ ]:
# Ruta absoluta al dataset crudo comprimido
DATASET_PATH = "/home/sanmartinofacundo/datasets/competencia_02_crudo.csv.gz"

In [ ]:
conn = duckdb.connect(database=':memory:')

conn.execute(f"""
    CREATE TABLE competencia_02_crudo AS
    SELECT * FROM read_csv_auto('{DATASET_PATH}')
""")
print("✅ Tabla creada en memoria")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Tabla creada en memoria


In [ ]:
start = time.time()
# Crear tabla etiquetada con la misma lógica SQL
conn.execute("""
    CREATE OR REPLACE TABLE competencia_02 AS
    WITH datos AS (
        SELECT *,
            LEAD(foto_mes, 1) OVER(PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_mas1,
            LEAD(foto_mes, 2) OVER(PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_mas2
        FROM competencia_02_crudo
    ),
    maximo_mes AS (
        SELECT MAX(foto_mes) AS maximo_foto_mes FROM competencia_02_crudo
    ),
    etiquetado AS (
        SELECT
            d.*,
            CASE
                WHEN d.foto_mes = m.maximo_foto_mes THEN NULL
                WHEN d.foto_mes_mas1 IS NULL THEN 'BAJA+1'
                WHEN d.foto_mes_mas2 IS NULL
                     AND d.foto_mes <> m.maximo_foto_mes
                     AND d.foto_mes <> m.maximo_foto_mes - 1 THEN 'BAJA+2'
                ELSE 'CONTINUA'
            END AS clase_ternaria
        FROM datos d
        CROSS JOIN maximo_mes m
    )
    SELECT * EXCLUDE (foto_mes_mas1, foto_mes_mas2)
    FROM etiquetado;
""")

end = time.time()
print(f"✅ Tabla 'competencia_02' creada en memoria en {end - start:.2f} segundos")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Tabla 'competencia_01' creada en memoria en 8.42 segundos


In [ ]:
# Ejecutar la consulta SQL en DuckDB y obtener un DataFrame
df_resumen = conn.execute("""
    SELECT 
        foto_mes, 
        clase_ternaria, 
        COUNT(*) AS cantidad
    FROM competencia_02
    GROUP BY foto_mes, clase_ternaria
    ORDER BY foto_mes, clase_ternaria
""").df()

# Mostrar resultado
df_resumen.head(15)


,foto_mes,clase_ternaria,cantidad
0,202101,BAJA+1,616
1,202101,BAJA+2,824
2,202101,CONTINUA,160087
3,202102,BAJA+1,830
4,202102,BAJA+2,1027
5,202102,CONTINUA,160298
6,202103,BAJA+1,1034
7,202103,BAJA+2,953
8,202103,CONTINUA,161122
9,202104,BAJA+1,954


In [ ]:
# Ejecutar la consulta PIVOT en DuckDB y obtener un DataFrame
df_pivot = conn.execute("""
    PIVOT competencia_02
    ON clase_ternaria
    USING count(numero_de_cliente)
    GROUP BY foto_mes
""").df()

# Mostrar resultado
df_pivot.head(36)

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,616,824,160087
1,202102,830,1027,160298
2,202103,1034,953,161122
3,202104,954,1131,161333
4,202105,1134,0,162783
5,202106,0,0,0


In [ ]:
# Guardar en bucket de GCP (b1)
dataset_path_local = "/home/sanmartinofacundo/dmeyf2025/competencia_02/datos/competencia_02.csv.gz"
conn.execute(f"""
    COPY competencia_02 
    TO '{dataset_path_local}' 
    (FORMAT CSV, HEADER, COMPRESSION GZIP);
""")

# Subir al bucket
!gsutil cp {dataset_path_local} gs://b1/
print("✅ Copiado a bucket b1 correctamente")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Archivo exportado a: c:\Users\Facundo San Martino\Desktop\dmeyf2025\competencia_02\target_feature_engineering\..\datos\competencia_01.csv
